Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [6]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    !pip install category_encoders==2.*

In [7]:
import numpy as np
import pandas as pd
import category_encoders as ce
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [8]:
def dadaWrangle( x):
    x = x.copy()

    x['latitude'] = x['latitude'].replace( -2e-08, 0)
    zerodCols = ['latitude', 'longitude', 
                 'construction_year', 
                 'gps_height','population']
    for cols in zerodCols:
        x[cols] = x[cols].replace( 0, np.nan)
        x[cols + '_MISSING'] = x[cols].isnull()
    
    x['date_recorded'] = pd.to_datetime( x['date_recorded'], 
                                        infer_datetime_format= True)
    x['year_recorded'] = x['date_recorded'].dt.year
    x['month_recorded'] = x['date_recorded'].dt.month
    x['day_recorded'] = x['date_recorded'].dt.day

    ## moved date stratification up so df.drop only needs to be called once

    x = x.drop( columns= ['quantity_group', 'payment_type',     ## duplicates
                          'recorded_by', 'id',                  ## unusable
                          'date_recorded'])                     ## superfluous

    x['age'] = ( x['year_recorded'] - x['construction_year'] )
    x['years_MISSING'] = x['age'].isnull()

    return x

dadaWrangle( train)

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,latitude_MISSING,longitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,year_recorded,month_recorded,day_recorded,age,years_MISSING
0,6000.0,Roman,1390.0,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,...,False,False,False,False,False,2011,3,14,12.0,False
1,0.0,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,...,False,False,False,False,False,2013,3,6,3.0,False
2,25.0,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,...,False,False,False,False,False,2013,2,25,4.0,False
3,0.0,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,...,False,False,False,False,False,2013,1,28,27.0,False
4,0.0,Action In A,NaN,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,...,False,False,True,True,True,2011,7,13,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,Germany Republi,1210.0,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,...,False,False,False,False,False,2013,5,3,14.0,False
59396,4700.0,Cefa-njombe,1212.0,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,...,False,False,False,False,False,2011,5,7,15.0,False
59397,0.0,NaN,NaN,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,...,False,False,True,True,True,2011,4,11,NaN,True
59398,0.0,Malec,NaN,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,...,False,False,True,True,True,2011,3,8,NaN,True


In [9]:
target = 'status_group'

## fallInLine.exe commencing...

tFeatures = train.drop( columns= [target, 'id'])
numFeatures = tFeatures.select_dtypes( include= 'number').columns.tolist()
cardiB = tFeatures.select_dtypes( exclude= 'number').nunique()
catFeatures = cardiB[ cardiB <= 50].index.tolist()

features = numFeatures + catFeatures

In [10]:
xTrain = train[ features]
yTrain = train[ target]

xTest = test[ features]

In [12]:
%%time
popeline = make_pipeline(
    ce.OneHotEncoder( use_cat_names= True), 
    SimpleImputer( strategy= 'mean'), 
    SelectKBest( f_classif, k= 32), 
    RandomForestClassifier( n_estimators= 100, n_jobs= -1, random_state= 16)
)

k = 3
scored = cross_val_score( popeline, xTrain, yTrain, cv= k, 
                          scoring= 'neg_mean_absolute_error')
print( 'MAE for {} folds: {:.5f}'.format( k, -scored) )

C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [42] are constant.
  UserWarning)
C:\Users\reall\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


ValueError: could not convert string to float: 'functional'

In [ ]:
-scores.mean()

In [ ]:
print('Model Hyperparameters:')
print(pipeline.named_steps['randomforestregressor'])